<a href="https://colab.research.google.com/github/Kerriea-star/TensorFlow-Customization/blob/main/Customization_basics_tensors_and_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here, you'll learn how to:

*   Import the required package.
*   Create and use tensors.
*   Use GPU acceleration.
*   Build a data pipeline with `tf.data.Dataset`



### Import TensorFlow

As of TensorFlow 2, eager execution is turned on by default. Eager execution enables a more interactive frontend to TensorFlow.

In [1]:
import tensorflow as tf